In [ ]:
#This script audits the PMU data stored on flywheel, either at the acquisition or session level.
#Inputs:
    #enrollment sheets pulled from axis, stored at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/
    #data from Flywheel acessed using the CLI 
#Outputs:
    #Binary audit of PMU data stored in DICOMS on Flywheel 

In [1]:
import flywheel
import pandas as pd
import re 
fw = flywheel.Client()

In [2]:
EFProj=fw.projects.find_first('label=EFR01')

/Users/krmurtha/anaconda3/lib/python3.7/site-packages/flywheel/flywheel.py:6274: UserWarning: Client version 14.6.6 does not match server version 16.4.3. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [3]:
#loop through subs and get sessions
subjects=EFProj.subjects()

sessions=[]
for s in subjects :
    tempsessions=s.sessions()
    sessions.extend(tempsessions)

In [5]:
#read in T1 enrollment
#adjust file path as necessary -- currently assuming parent directory is saturn folder referenced in first block 
axis_t1=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_enroll_t1.csv',dtype=str)
axis_t1=axis_t1.drop(columns=['scan_1_date'])
#and T2
axis_t2=pd.read_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/axis_enroll_t2.csv',dtype=str)
axis_t2=axis_t2.drop(columns=['scan_2_date'])

In [6]:
#reformat for list of scan IDs 
t1_scan=axis_t1['scan_id_timepoint_1']
t1_scan=t1_scan.tolist()
t1_scan = [str(t) for t in t1_scan]
#and again
t2_scan=axis_t2['scan_id_t2']
t2_scan=t2_scan.tolist()
t2_scan = [str(t) for t in t2_scan]

In [17]:
#get a list of all log files attached at the session level of flywheel 
has_rest1_pmu=[]
has_frac_pmu=[]
has_rest2_pmu=[]

for ses in sessions:
    sesid=ses.label
    #print(sesid)
    acq=ses.acquisitions()
    for a in acq:
        if 'func_task-rest_run-01_PMU' in a.label or 'ABCD_fMRI_rest_PMU' in a.label:
            has_rest1_pmu.append(sesid)
        if 'func_task-fracnoback_run-02_PMU' in a.label or 'ABCD_fMRI_frac-no-back-run1_PMU' in a.label:
            has_frac_pmu.append(sesid)
        if 'func_task-rest_run-03_PMU' in a.label or 'ABCD_fMRI_rest_PMU' in a.label:
            has_rest2_pmu.append(sesid)

In [43]:
print(len(has_rest1_pmu))
print(len(has_frac_pmu))
print(len(has_rest2_pmu))

177
166
153


In [27]:
t1_audit = pd.DataFrame(t1_scan, columns =['scanid'])

In [28]:
#fill in columns for audit sheet 
rest1_pmu=[]
frac_pmu=[]
rest2_pmu=[]

for val in t1_audit.values:
    if val in has_rest1_pmu:
        rest1_pmu.append(1)
    if val not in has_rest1_pmu:
        rest1_pmu.append(0)
    if val in has_frac_pmu:
        frac_pmu.append(1)
    if val not in has_frac_pmu:
        frac_pmu.append(0)
    if val in has_rest2_pmu:
        rest2_pmu.append(1)
    if val not in has_rest2_pmu:
        rest2_pmu.append(0)

In [30]:
#fill in audit 
t1_audit['rest1_pmu']=rest1_pmu
t1_audit['frac_pmu']=frac_pmu 
t1_audit['rest2_pmu']=rest2_pmu

In [33]:
t1_audit.head(n=5)

,scanid,rest1_pmu,frac_pmu,rest2_pmu
0,11012,1,1,1
1,10960,0,0,0
2,10959,1,0,1
3,11138,1,1,1
4,11000,0,0,0


In [41]:
#write out T1 file 
t1_audit.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/audits/EF_T1_PMU_audit.csv', sep = ',', index=False)

In [35]:
#and again for T2
t2_audit = pd.DataFrame(t2_scan, columns =['scanid'])

In [36]:
rest1_pmu=[]
frac_pmu=[]
rest2_pmu=[]

for val in t2_audit.values:
    if val in has_rest1_pmu:
        rest1_pmu.append(1)
    if val not in has_rest1_pmu:
        rest1_pmu.append(0)
    if val in has_frac_pmu:
        frac_pmu.append(1)
    if val not in has_frac_pmu:
        frac_pmu.append(0)
    if val in has_rest2_pmu:
        rest2_pmu.append(1)
    if val not in has_rest2_pmu:
        rest2_pmu.append(0)

In [39]:
t2_audit['rest1_pmu']=rest1_pmu
t2_audit['frac_pmu']=frac_pmu 
t2_audit['rest2_pmu']=rest2_pmu

In [40]:
t2_audit.head(n=5)

,scanid,rest1_pmu,frac_pmu,rest2_pmu
0,11657,1,1,1
1,11831,1,1,1
2,11594,1,1,1
3,11659,1,1,1
4,11612,1,1,1


In [42]:
t2_audit.to_csv('/Volumes/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/audits/EF_T2_PMU_audit.csv', sep = ',', index=False)